In [ ]:
import numpy as np
import pandas as pd
import utils
import scoring
import catboost
from sklearn.model_selection import StratifiedKFold, KFold

## public test

In [ ]:
full_train = pd.read_csv("data/test_public_v2.csv", dtype=dtypes, index_col='id')

In [ ]:
closest_hits_features = pd.read_csv('more_features_fixed_test.csv',dtype=np.float32)

In [ ]:
train_concat = pd.concat(
    [full_train.loc[:, utils.SIMPLE_FEATURE_COLUMNS],
     closest_hits_features], axis=1)

## private test

In [ ]:
#full_train = pd.read_csv("data/test_private_v3_track_1.csv", dtype=dtypes, index_col='id')

In [ ]:
#closest_hits_features = pd.read_csv('more_features_fixed_test_private.csv',dtype=np.float32)

In [ ]:
#train_concat = pd.concat(
#    [full_train.loc[:, utils.SIMPLE_FEATURE_COLUMNS],
#     closest_hits_features], axis=1)

## additional features

In [ ]:
for i in range(4):
    train_concat[f'plastina_{i}'] = np.sqrt(train_concat[f'MatchedHit_X[{i}]'] ** 2 + train_concat[f'MatchedHit_Y[{i}]'] ** 2 + train_concat[f'MatchedHit_Z[{i}]']**2)

In [ ]:
for i in range(4):
    train_concat.loc[(train_concat[f'MatchedHit_X[{i}]']==-9999)|(train_concat[f'MatchedHit_Y[{i}]']==-9999)|(train_concat[f'MatchedHit_Z[{i}]']==-9999),f'plastina_{i}'] = -9999

In [ ]:
for i in range(8):
    train_concat[f'arr_{i}'] = train_concat[f'{i}'] / train_concat[f'{16+i}']**2

In [ ]:
train_concat['angle'] = np.arccos(train_concat['PT'] / train_concat['P'])

In [ ]:
for i in range(4):
    train_concat[f'size_clusters_{i}']=train_concat[f'ncl[{i}]']*train_concat[f'avg_cs[{i}]']

In [ ]:
for i in range(4):
    train_concat[f'v_{i}'] = train_concat[f'plastina_{i}'] / train_concat[f'MatchedHit_T[{i}]']

## evaluate catboost model mix

In [ ]:
model = catboost.CatBoostClassifier(loss_function='Logloss',
                                    l2_leaf_reg=20,
                                    border_count=200,
                                    max_depth=5,
                                    bootstrap_type='No', 
                                    random_strength=0, 
                                    iterations=3500, 
                                    learning_rate=0.05, 
                                    verbose=False,
                                    thread_count=4, 
                                    task_type='GPU', 
                                    random_state=12,
                                    rsm=1,
)

In [ ]:
predictions = None
for i in range(5):
    model.load_model(f'fold_{i}.cbm')
    val_predictions = model.predict_proba(train_concat.values)[:, 1]
    if i > 0:
        predictions += val_predictions
    else:
        predictions = val_predictions
predictions /= 5

## submission

In [ ]:
pd.DataFrame(data={"prediction": predictions}, index=full_train.index).to_csv(
    "submission.csv", index_label="id")